In [2]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
import open3d as o3d
from find_box import BoxSegmentator, BoxPoseEstimator
from realsense_camera import RealsenseCamera
from camera_utils import stream
from copy import deepcopy

def show(image):
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# initialization

camera = RealsenseCamera(enable_depth=True)

camera.start()
K = camera.get_K()
D = camera.get_D()
depth_scale = camera.get_depth_scale()
camera.stop()

box_segmentator = BoxSegmentator(erosion_size=10)
box_pose_estimator = BoxPoseEstimator(
    edges_sizes=np.array([0.11, 0.05, 0.022]), edge_points_per_cm=7,
    voxel_size=0.003, depth_scale=depth_scale, K=K, D=D, mutual_filter=False,
    global_max_dist=0.04, global_checker_max_dist=0.01, max_dist=0.007)

In [24]:
# test box segmentation

camera.start()
stream(camera)  # waits for a key to get image
image = camera()
camera.stop()

mask = box_segmentator.segment_box(image)
masked_image = image.copy()
masked_image[mask == 0] = 0
show(masked_image)

In [ ]:
# estimate box pose

camera.start()
stream(camera)  # waits for a key to get image
image, depth = camera.read(read_color=True, read_depth=True)
camera.stop()

mask = box_segmentator.segment_box(image)
global_reg, reg, extracted_box_pc, extracted_box_pc_down = \
    box_pose_estimator.estimate_box_pose(mask, depth)
box_pose = reg.transformation

In [26]:
# visualize point clouds

transformed_box_pc = deepcopy(box_pose_estimator.box_pc)
transformed_box_pc.transform(box_pose)
transformed_box_pc.paint_uniform_color(np.array([1, 0, 0]))
extracted_box_pc.paint_uniform_color(np.array([0, 1, 0]))
o3d.visualization.draw_geometries([transformed_box_pc, extracted_box_pc])

In [37]:
# visualize box frame

force_z_axis_up = False
if force_z_axis_up:
    if box_pose[1, 2] > 0:
        correction = np.eye(4)
        correction[0:3, 0:3], _ = cv2.Rodrigues(np.array([np.pi, 0., 0.]))
        pose = np.matmul(box_pose, correction)
else:
    pose = box_pose

draw = image.copy()
rvec, _ = cv2.Rodrigues(pose[0:3, 0:3])
tvec = box_pose[0:3, 3]
cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.07)
show(draw)